In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
pd.set_option("display.max_colwidth", -1)

In [4]:
data = pd.read_csv('winemag-data_first150k.csv', delimiter=",", encoding="utf-8")
data = data.iloc[:,1:]

In [5]:
data.head(3)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,"Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 g/L of residual sugar.",Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley


In [6]:
data.drop(["region_1", "region_2"], axis=1, inplace=True)

In [7]:
data.isnull().sum()

country        5    
description    0    
designation    45735
points         0    
price          13695
province       5    
variety        0    
winery         0    
dtype: int64

In [8]:
set(data.country)

{'Albania',
 'Argentina',
 'Australia',
 'Austria',
 'Bosnia and Herzegovina',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Egypt',
 'England',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Hungary',
 'India',
 'Israel',
 'Italy',
 'Japan',
 'Lebanon',
 'Lithuania',
 'Luxembourg',
 'Macedonia',
 'Mexico',
 'Moldova',
 'Montenegro',
 'Morocco',
 'New Zealand',
 'Portugal',
 'Romania',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'South Africa',
 'South Korea',
 'Spain',
 'Switzerland',
 'Tunisia',
 'Turkey',
 'US',
 'US-France',
 'Ukraine',
 'Uruguay',
 nan}

In [9]:
data.isnull().sum()

country        5    
description    0    
designation    45735
points         0    
price          13695
province       5    
variety        0    
winery         0    
dtype: int64

In [10]:
from collections import Counter

In [11]:
Counter(data.designation)

Counter({nan: 45735,
         'Blu Giovello': 3,
         'Vineyard X': 1,
         'Zanna Riserva': 2,
         'Vino Rosso di Santa Barbara Red Wine': 1,
         'Nantua Vineyard': 4,
         'Grand Clos des Epenots Premier Cru': 1,
         'Purus': 2,
         'T26 Frauen Weingarten': 2,
         'Château Reserve Méthode Champenoise Blanc de Blancs Brut': 1,
         'Ockfener Bockstein': 1,
         'Bodacious': 4,
         'Trincadeira-Aragonez': 4,
         'Pinot Grigio Grecanico': 1,
         'Badarina': 4,
         "L'Autre": 7,
         'Herrenweg de Turckheim': 2,
         'Two Worlds': 1,
         'Old Vines Estate': 2,
         'Grüner Veltliner Kirchlissen': 3,
         'Les Tourelles de Longueville Barrel Sample': 2,
         'Hermada': 2,
         'Christie Vineyard': 4,
         'Grand Cru Kessler': 3,
         'Cedarman': 1,
         'Beachfront': 5,
         'Hillside Vineyard': 7,
         'Veleta Nolados': 2,
         'Opera Prima Pink Moscato': 1,
         'San

In [12]:
# Country
data.country[data.winery=='Chilcas']="Chile"
data.country[data.winery=="Tsililis"] = "Greece"
data.drop([1440], axis=0, inplace=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
data.loc[(data.country=="Greece") & (data.winery=="Tsililis") & data.province.isnull(),'province'] = "Greece"
data.loc[(data.country=="Chile") & (data.winery=="Chilcas") & (data.variety=="Pinot Noir"), "province"] = "Maule Valley"

In [14]:
data

,country,description,designation,points,price,province,variety,winery
0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.",Martha's Vineyard,96,235.0,California,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Tinta de Toro,Bodega Carmen Rodríguez
2,US,"Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 g/L of residual sugar.",Special Selected Late Harvest,96,90.0,California,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032.",Reserve,96,65.0,Oregon,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named after the highest point in the vineyard at 1200 feet. It has structure, density and considerable acidity that is still calming down. With 18 months in wood, the wine has developing an extra richness and concentration. Produced by the Tari family, formerly of Château Giscours in Margaux, it is a wine made for aging. Drink from 2020.",La Brûlade,95,66.0,Provence,Provence red blend,Domaine de la Bégude
5,Spain,"Deep, dense and pure from the opening bell, this Toro is a winner. Aromas of dark ripe black fruits are cool and moderately oaked. This feels massive on the palate but sensationally balanced. Flavors of blackberry, coffee, mocha and toasty oak finish spicy, smooth and heady. Drink this exemplary Toro through 2023.",Numanthia,95,73.0,Northern Spain,Tinta de Toro,Numanthia
6,Spain,"Slightly gritty black-fruit aromas include a sweet note of pastry along with a hint of prune. Wall-to-wall saturation ensures that all corners of one's mouth are covered. Flavors of blackberry, mocha and chocolate are highly impressive and expressive, while this settles nicely on a long finish. Drink now through 2024.",San Román,95,65.0,Northern Spain,Tinta de Toro,Maurodos
7,Spain,"Lush cedary black-fruit aromas are luxe and offer notes of marzipan and vanilla. This bruiser is massive and tannic on the palate, but still lush and friendly. Chocolate is a key flavor, while baked berry and cassis flavors are hardly wallflowers. On the finish, this is tannic and deep as a sea trench. Drink this saturated black-colored Toro through 2023.",Carodorum Único Crianza,95,110.0,Northern Spain,Tinta de Toro,Bodega Carmen Rodríguez
8,US,"This re-named vineyard was formerly bottled as deLancellotti. You'll find striking minerality underscoring chunky black fruits. Accents of citrus and graphite comingle, with exceptional midpalate concentration. This is a wine to cellar, though it is already quite enjoyable. Drink now through 2030.",Silice,95,65.0,Oregon,Pinot Noir,Bergström
9,US,"The producer sources from two blocks of the vineyard for this wine—one at a high elevation, which contributes bright acidity. Crunchy cranberry, pomegranate and orange peel flavors surround silky, succulent layers of texture that present as flesh

In [15]:
data.loc[(data.country=="Italy") & (data.winery=="Abbadia Ardenga") & (data.variety=="Sangiovese"), "price"] = 65.0

In [16]:
data.loc[(data.country=="France") & (data.winery=="Domaine Vacheron") & (data.variety=="Sauvignon Blanc") & (data.designation=="Les Romains"), "price"] = 46.666666666666664

In [17]:
data.loc[(data.country=="France") & (data.winery=="Domaine Vacheron") & (data.variety=="Sauvignon Blanc") & (data.price.isnull()), "price"] = 39.428571428571423

In [18]:
data.loc[(data.country=="France") & (data.winery=="Domaine Vacheron") & (data.price.isnull()), "price"] = 41.18796992481203

In [19]:
data.loc[(data.country=="France") & (data.province=="Bordeaux") & (data.winery=="Les Hauts de Tousquiron"), "price"] = 28.0

In [20]:
data.loc[(data.country=="France") & (data.province=="Bordeaux") & (data.winery=="Château Ferran"), "price"] = 42.574539992489676

In [21]:
data.loc[(data.country=="France") & (data.province=="Bordeaux") & (data.winery=="Château de Portets") & (data.price.isnull()), "price"] = 22.0

In [22]:
data.loc[(data.country=="France") & (data.province=="Bordeaux") & (data.winery=="Château Lafargue"), "price"] = 42.566819714818948

In [23]:
data.loc[(data.country=="Italy") & (data.price.isnull()), "price"] = 37.549374500931592

In [24]:
data.loc[(data.country=="US") & (data.price.isnull()), "price"] = 33.653808397302818

In [25]:
data.loc[(data.country=="Spain") & (data.price.isnull()), "price"] = 27.048529411764704

In [26]:
data.loc[(data.country=="France") & (data.price.isnull()), "price"] = 45.609149427629319

In [27]:
data.loc[(data.country=="Austria") & (data.price.isnull()), "price"] = 31.192106322996377

In [28]:
data.loc[(data.country=="Portugal") & (data.price.isnull()), "price"] = 26.332614942528735

In [29]:
data.loc[(data.country=="New Zealand") & (data.price.isnull()), "price"] = 24.173289902280132

In [30]:
data.loc[(data.country=="Argentina") & (data.price.isnull()), "price"] = 20.794880973688922

In [31]:
data.loc[(data.country=="Germany") & (data.price.isnull()), "price"] = 39.011077971878997

In [32]:
data.loc[(data.country=="Australia") & (data.price.isnull()), "price"] = 31.258479771148345

In [33]:
data.loc[(data.country=="Chile") & (data.price.isnull()), "price"] = 19.342520367481367

In [34]:
data.loc[(data.country=="Israel") & (data.price.isnull()), "price"] = 31.304918032786887

In [35]:
data.loc[(data.country=="South Africa") & (data.price.isnull()), "price"] = 21.130531962449709

In [36]:
data.loc[(data.country=="Slovenia") & (data.price.isnull()), "price"] = 28.061728395061728

In [37]:
data.loc[(data.country=="Turkey") & (data.price.isnull()), "price"] = 25.800000000000001

In [38]:
data.loc[(data.country=="Greece") & (data.price.isnull()), "price"] = 21.742268041237114

In [39]:
data.loc[(data.country=="England") & (data.price.isnull()), "price"] = 47.5

In [40]:
data.loc[(data.country=="Uruguay") & (data.price.isnull()), "price"] = 25.847058823529412

In [41]:
data.loc[(data.country=="Croatia") & (data.price.isnull()), "price"] = 23.108433734939759

In [42]:
data.loc[(data.country=="Hungary") & (data.price.isnull()), "price"] = 44.204347826086959

In [43]:
data.loc[(data.country=="Canada") & (data.price.isnull()), "price"] = 34.628865979381445

In [49]:
data.loc[(data.price.isnull()), "price"] = 33.703910449195497